In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.core.utils.openbb_helpers import obb_login
from openbb import obb
from humbldata.core.utils.env import Env
import polars as pl

# obb_login()
obb.account.login(pat=Env().OBB_PAT, remember_me=True)

In [ ]:

from humbldata.toolbox.technical.mandelbrot_channel.model import (
    calc_mandelbrot_channel,
    calc_mandelbrot_channel_historical,
)


data = (
    obb.equity.price.historical(
        ["AAPL"],
        provider="yfinance",
        start_date="2020-01-01",
        end_date="2024-01-01",
        adjustment="splits_and_dividends",
    ).to_polars()
).drop(["dividends", "stock_splits"]).with_columns(pl.lit("AAPL").alias("symbol"))

mandelbrot = calc_mandelbrot_channel_historical(  # noqa: ERA001
        data,
        window="1m",
        rv_adjustment=True,
        rv_method="std",
        rv_grouped_mean=False,
        rs_method="RS",
        live_price=False,
    ).collect()
mandelbrot

# Setting up UserTable

In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.core.standard_models.portfolio.analytics.user_table import UserTableQueryParams
from humbldata.core.utils.openbb_helpers import get_latest_price


symbols = UserTableQueryParams(symbol="AAPL, MSFT, NVDA").symbol
symbols



get_latest_price(symbols)

# Tesing Async


## Get Latest Price Async

In [ ]:
import asyncio
import logging
from typing import List

import polars as pl
from openbb import obb

from humbldata.core.utils.constants import OBB_EQUITY_PRICE_QUOTE_PROVIDERS

async def get_latest_price_async(
    symbol: str | List[str] | pl.Series,
    provider: OBB_EQUITY_PRICE_QUOTE_PROVIDERS | None = "yfinance",
) -> pl.LazyFrame:
    """
    Asynchronous version of get_latest_price.

    Context: Core || Category: Utils || Subcategory: OpenBB Helpers || **Command: get_latest_price_async**.

    Queries the latest stock price data for the given symbol(s) using the
    specified provider asynchronously. Defaults to YahooFinance (`yfinance`) if no provider is
    specified. Returns a LazyFrame with the stock symbols and their latest prices.

    Parameters
    ----------
    symbol : str | List[str] | pl.Series
        The stock symbol(s) to query for the latest price. Accepts a single
        symbol, a list of symbols, or a Polars Series of symbols.
    provider : OBB_EQUITY_PRICE_QUOTE_PROVIDERS, optional
        The data provider for fetching stock prices. Default is `yfinance`.

    Returns
    -------
    pl.LazyFrame
        A Polars LazyFrame with columns for the stock symbols ('symbol') and
        their latest prices ('recent_price').
    """
    logging.getLogger("openbb_terminal.stocks.stocks_model").setLevel(
        logging.CRITICAL
    )

    loop = asyncio.get_event_loop()
    result = await loop.run_in_executor(
        None, lambda: obb.equity.price.quote(symbol, provider=provider)
    )

    return (
        result.to_polars()
        .lazy()
        .select(["symbol", "last_price"])
        .rename({"last_price": "recent_price"})
    )

In [ ]:
import asyncio
import time
from pathlib import Path

import polars as pl
from humbldata.core.utils.openbb_helpers import get_latest_price
import nest_asyncio
nest_asyncio.apply()

async def run_async_version(symbols):
    return await get_latest_price_async(symbols)

def run_sync_version(symbols):
    return get_latest_price(symbols)

async def main():
    symbols = ["AAPL", "GOOGL", "MSFT", "AMZN", "FB"]

    # Test synchronous version
    start_time = time.time()
    sync_result = run_sync_version(symbols)
    sync_time = time.time() - start_time
    print(f"Synchronous execution time: {sync_time:.4f} seconds")
    print("Synchronous result:")
    print(sync_result.collect())

    print("\n" + "="*50 + "\n")

    # Test asynchronous version
    start_time = time.time()
    async_result = await run_async_version(symbols)
    async_time = time.time() - start_time
    print(f"Asynchronous execution time: {async_time:.4f} seconds")
    print("Asynchronous result:")
    print(async_result.collect())

    print("\n" + "="*50 + "\n")
    print(f"Time difference: {sync_time - async_time:.4f} seconds")
    print(f"Speedup factor: {sync_time / async_time:.2f}x")

# Modified execution part
loop = asyncio.get_event_loop()
loop.run_until_complete(main())

## Get Sector Async

In [ ]:
import asyncio
from typing import List

import polars as pl
from openbb import obb

from humbldata.core.utils.constants import OBB_EQUITY_PROFILE_PROVIDERS

async def get_sector_async(
    symbols: str | List[str] | pl.Series,
    provider: OBB_EQUITY_PROFILE_PROVIDERS | None = "yfinance",
) -> pl.LazyFrame:
    """
    Asynchronous version of get_sector.

    Context: Core || Category: Utils || Subcategory: OpenBB Helpers || **Command: get_sector_async**.

    Retrieves the sector information for the given stock symbol(s) using OpenBB's equity profile data asynchronously.

    Parameters
    ----------
    symbols : str | List[str] | pl.Series
        The stock symbol(s) to query for sector information. Accepts a single
        symbol, a list of symbols, or a Polars Series of symbols.
    provider : str | None, optional
        The data provider to use for fetching sector information. If None, the default
        provider will be used.

    Returns
    -------
    pl.LazyFrame
        A Polars LazyFrame with columns for the stock symbols ('symbol') and
        their corresponding sectors ('sector').

    Notes
    -----
    This function uses OpenBB's equity profile data to fetch sector information.
    It returns a lazy frame for efficient processing, especially with large datasets.
    """
    loop = asyncio.get_event_loop()
    result = await loop.run_in_executor(
        None, lambda: obb.equity.profile(symbols, provider=provider)
    )

    return result.to_polars().lazy().select(["symbol", "sector"])

In [ ]:
import asyncio
import time
from pathlib import Path

import polars as pl
from humbldata.core.utils.openbb_helpers import get_equity_sector

async def run_async_version(symbols):
    return await get_sector_async(symbols)

def run_sync_version(symbols):
    return get_equity_sector(symbols)

async def main():
    symbols = ["XLK", "FXE", "FXF", "AMZN", "FB"]

    # Test synchronous version
    start_time = time.time()
    sync_result = run_sync_version(symbols)
    sync_time = time.time() - start_time
    print(f"Synchronous execution time: {sync_time:.4f} seconds")
    print("Synchronous result:")
    print(sync_result.collect())

    print("\n" + "="*50 + "\n")

    # Test asynchronous version
    start_time = time.time()
    async_result = await run_async_version(symbols)
    async_time = time.time() - start_time
    print(f"Asynchronous execution time: {async_time:.4f} seconds")
    print("Asynchronous result:")
    print(async_result.collect())

    print("\n" + "="*50 + "\n")
    print(f"Time difference: {sync_time - async_time:.4f} seconds")
    print(f"Speedup factor: {sync_time / async_time:.2f}x")

# Use this for regular Python scripts
# if __name__ == "__main__":
#     loop = asyncio.get_event_loop()
#     loop.run_until_complete(main())

# Use this for Jupyter Notebooks
import nest_asyncio
nest_asyncio.apply()
await main()

## Get ETF Category Async

In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.core.utils.openbb_helpers import aget_etf_sector
import asyncio

stocks = pl.Series([
    "GOOGL",  # Communication Services
    "AMZN",   # Consumer Discretionary
    "KO",     # Consumer Staples
    "XOM",    # Energy
    "JPM",    # Financials
    "JNJ",    # Health Care
    "HON",    # Industrials
    "AAPL",   # Information Technology
    "LIN",    # Materials
    "PLD",    # Real Estate
    "NEE"     # Utilities
])
test = (await aget_etf_sector(symbols=stocks, provider="yfinance")).collect()

In [ ]:
test

In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.core.utils.openbb_helpers import aget_equity_sector
import polars as pl

stocks = pl.Series([
    "GOOGL",  # Communication Services
    "AMZN",   # Consumer Discretionary
    "KO",     # Consumer Staples
    "XOM",    # Energy
    "JPM",    # Financials
    "JNJ",    # Health Care
    "HON",    # Industrials
    "AAPL",   # Information Technology
    "LIN",    # Materials
    "PLD",    # Real Estate
    "NEE"     # Utilities
])


test2 = await aget_equity_sector(symbols=stocks, provider="yfinance")
test2.collect()

In [ ]:
obb.etf.info(symbol='XLK', provider='yfinance').to_polars()

In [ ]:
obb.equity.profile(["XLK", "XLE"], provider='yfinance').to_polars()

In [ ]:
df = pl.DataFrame({"symbol": ["XLK", "XLE"], "sector": ["Tech", "Equity Energy"]})


df.with_columns(pl.col("sector").str.replace_all("Equity Energy", "Energy"))

In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.core.utils.constants import EQUITY_SECTOR_MAPPING
from humbldata.portfolio.analytics.user_table.helpers import normalize_sector


df.with_columns(pl.col("sector").replace(EQUITY_SECTOR_MAPPING))

## Test aget_sector_filter

In [79]:
%load_ext autoreload
%autoreload 2

from humbldata.portfolio.analytics.user_table.helpers import aget_sector_filter


(await aget_sector_filter(symbols=["XLU", "XLE", "XLF", "XLC", "KO"])).collect()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


symbol,sector
str,str
"""KO""","""Consumer Defensive"""
"""XLU""","""Utilities"""
"""XLE""","""Energy"""
"""XLF""","""Financials"""
"""XLC""","""Communication Services"""


# Test aggregate_user_table_data

In [83]:
from humbldata.portfolio.analytics.user_table.helpers import aggregate_user_table_data


(await aggregate_user_table_data(symbols=["AAPL", "XLE"])).collect()

symbol,recent_price,sector
str,f64,str
"""AAPL""",209.07,"""Technology"""
"""XLE""",null,"""Energy"""
